In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import math
import os
import copy
import time

from tempfile import TemporaryDirectory
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

from sklearn.cluster import KMeans

In [3]:
songStrings = numpy.array([
    "CCGGAAGFFEEDDCGGFFEEDGGFFEEDCCGGAAGFFEEDDC",
    "ABCDEFABCDEFABCDEFABCDEFABCDEFABCDEFABCDEF",
    "ABACADAEAFABEFADECBABCFEDEFABCADEBACADFABE",
    "DBCACBCFFDCEFFEFCDDEFEBEACFECBBBCBECBFDAFB",
    "ABEBCAEFCDFFBCBDBBBCEDCBFBFFECBCEBCAAFFADB",
    "BEEFBAFDAEAAEFDBDFDEFCACEBCCDACEACACEEDBAA",
    "BFEBFEEBDBCFEAACAAAFDFCBFBFEAACFFCAABCEDDC",
    "BADDFFEADBEDFDFBEBCCADEFDEABBFDEFFEBEEFDEF",
    "ABFFEDBDBFECEDEAEBBEECFDDAEDCDBBFCADADBBCF",
    "DFBCEBDAADAAFCDACADDAFFACDCFCCDDDCFBEBBDED",
    "CCFBEFDDCBFDADDBFBCCEEABAFAAAEDCDCEAEFBFCD",
    "EBADFFAAFADDDABEABBDFDCAFBCDEEBBBECDDFEEAE",
    "AFADDFEFADDBCDCFEEFCAEEEDFFEDBCADBBDBAEFCD"])
notes=list("ABCDEFGH")
print(notes)
chord=[[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7]]
allSongs=list()
for songString in songStrings:
    song=[]
    for note in list(songString):
        row=[0]*8
        row[notes.index(note)]=1
        song.append(chord[notes.index(note)])
    allSongs.append(song)
for song in allSongs:
    print(song)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
[[2], [2], [6], [6], [0], [0], [6], [5], [5], [4], [4], [3], [3], [2], [6], [6], [5], [5], [4], [4], [3], [6], [6], [5], [5], [4], [4], [3], [2], [2], [6], [6], [0], [0], [6], [5], [5], [4], [4], [3], [3], [2]]
[[0], [1], [2], [3], [4], [5], [0], [1], [2], [3], [4], [5], [0], [1], [2], [3], [4], [5], [0], [1], [2], [3], [4], [5], [0], [1], [2], [3], [4], [5], [0], [1], [2], [3], [4], [5], [0], [1], [2], [3], [4], [5]]
[[0], [1], [0], [2], [0], [3], [0], [4], [0], [5], [0], [1], [4], [5], [0], [3], [4], [2], [1], [0], [1], [2], [5], [4], [3], [4], [5], [0], [1], [2], [0], [3], [4], [1], [0], [2], [0], [3], [5], [0], [1], [4]]
[[3], [1], [2], [0], [2], [1], [2], [5], [5], [3], [2], [4], [5], [5], [4], [5], [2], [3], [3], [4], [5], [4], [1], [4], [0], [2], [5], [4], [2], [1], [1], [1], [2], [1], [4], [2], [1], [5], [3], [0], [5], [1]]
[[0], [1], [4], [1], [2], [0], [4], [5], [2], [3], [5], [5], [1], [2], [1], [3], [1], [1], [1], [2], [4], [3], [2],

In [34]:
def get_batch(source, i,songLength=42):# -> Tuple[Tensor, Tensor]
    song=numpy.floor(i/songLength).astype(int)
    start=i%songLength
    #print(song,start)
    data=[]
    target=[]
    for pos in range(start,start+songLength):
        data.append(source[song][pos%songLength])
        target+=source[song][(pos+1)%songLength]
    return data, target

d,t=get_batch(allSongs,6)
print(d)
print(t)

[[6], [5], [5], [4], [4], [3], [3], [2], [6], [6], [5], [5], [4], [4], [3], [6], [6], [5], [5], [4], [4], [3], [2], [2], [6], [6], [0], [0], [6], [5], [5], [4], [4], [3], [3], [2], [2], [2], [6], [6], [0], [0]]
[5, 5, 4, 4, 3, 3, 2, 6, 6, 5, 5, 4, 4, 3, 6, 6, 5, 5, 4, 4, 3, 2, 2, 6, 6, 0, 0, 6, 5, 5, 4, 4, 3, 3, 2, 2, 2, 6, 6, 0, 0, 6]


In [20]:
n= 5
length= 42

song=numpy.floor(87/(length)).astype(int)
print(song)

print(%length)

2
2


In [21]:
class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor, verbose=False) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        verbose=False
        if verbose:
            print(src.shape)
            figure()
            imshow(src.detach().numpy().reshape((42,20)))
        src = self.pos_encoder(src)
        if verbose:
            print(src.shape)
            figure()
            imshow(src.detach().numpy().reshape((42,20)))
        output = self.transformer_encoder(src, src_mask)
        if verbose:
            print(output.shape)
            figure()
            imshow(output.detach().numpy().reshape((42,20)))
        output = self.decoder(output)
        if verbose:
            print(output.shape)
            figure()
            imshow(output.detach().numpy().reshape((42,6)))
        return output


def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [22]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [24]:
def train(model: nn.Module,train_data) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(train_data) // bptt
    #for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
    who=numpy.array(list(range(bptt)))
    numpy.random.shuffle(who)
    for batch, i in enumerate(range(0, len(train_data) - 1, bptt)):
        data, targets = get_batch(train_data, who[i])
        seq_len = len(data)
        if seq_len != bptt:  # only on last batch
            src_mask = src_mask[:seq_len, :seq_len]
        output = model(torch.tensor(data,dtype=torch.long), src_mask)
        loss = criterion(output.view(-1, ntokens), torch.tensor(targets,dtype=torch.long))

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, len(eval_data) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            seq_len = len(data)
            if seq_len != bptt:
                src_mask = src_mask[:seq_len, :seq_len]
            output = model(torch.tensor(data,dtype=torch.long), src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += seq_len * criterion(output_flat, torch.tensor(targets,dtype=torch.long)).item()
    return total_loss / (len(eval_data) - 1)

In [43]:
x = range(0, len(allSongs) - 1, bptt)
print(x)

for batch, i in enumerate(range(0, 12, 210)):
    print(i)

range(0, 12, 210)
0


In [35]:
nrOfSongs=5
bptt = 42*nrOfSongs

#original
#ntokens = 8  # size of vocabulary
#emsize = 20  # embedding dimension
#d_hid = 20  # dimension of the feedforward network model in nn.TransformerEncoder
#nlayers = 2  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
#nhead = 4  # number of heads in nn.MultiheadAttention
#dropout = 0.03  # dropout probability

ntokens = 8  # size of vocabulary
emsize = 100  # embedding dimension
d_hid = 20  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 4  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 10  # number of heads in nn.MultiheadAttention
dropout = 0.03  # dropout probability

criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.999)


epochs=10000
W=[]
Os=[]
phi=[]
phiSamples=[]
Wsamples=[]
for epoch in range(1, epochs + 1):
    train(model,allSongs)
    
    model.eval()
    A=[]
    B=[]
    for song in range(nrOfSongs):
        data,target=get_batch(allSongs,42*song)
        src_mask = generate_square_subsequent_mask(bptt).to(device)
        output = model(torch.tensor(data,dtype=torch.long),src_mask,verbose=True)
        #torch.save(model,"models/DB_tf_epoch_{0}.model".format(epoch))

        
        for i in range(42):
            A.append(numpy.argmax(output.detach().numpy()[i]))
            B.append(target[i])
    score=(numpy.array(A)==numpy.array(B)).mean()
    W.append(score)
    if epoch%10==0:
        print(epoch,W[-1])
    #val_loss = evaluate(model, DaisyBell)
    #val_ppl = math.exp(val_loss)
    #print(f'| end of epoch {epoch:3d} | ' f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')

    #if val_loss < best_val_loss:
        #best_val_loss = val_loss
    scheduler.step()
plot(W)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


RuntimeError: The shape of the 2D attn_mask is torch.Size([210, 210]), but should be (42, 42).

In [24]:
allSongs

[[[2],
  [2],
  [6],
  [6],
  [0],
  [0],
  [6],
  [5],
  [5],
  [4],
  [4],
  [3],
  [3],
  [2],
  [6],
  [6],
  [5],
  [5],
  [4],
  [4],
  [3],
  [6],
  [6],
  [5],
  [5],
  [4],
  [4],
  [3],
  [2],
  [2],
  [6],
  [6],
  [0],
  [0],
  [6],
  [5],
  [5],
  [4],
  [4],
  [3],
  [3],
  [2]],
 [[0],
  [1],
  [2],
  [3],
  [4],
  [5],
  [0],
  [1],
  [2],
  [3],
  [4],
  [5],
  [0],
  [1],
  [2],
  [3],
  [4],
  [5],
  [0],
  [1],
  [2],
  [3],
  [4],
  [5],
  [0],
  [1],
  [2],
  [3],
  [4],
  [5],
  [0],
  [1],
  [2],
  [3],
  [4],
  [5],
  [0],
  [1],
  [2],
  [3],
  [4],
  [5]],
 [[0],
  [1],
  [0],
  [2],
  [0],
  [3],
  [0],
  [4],
  [0],
  [5],
  [0],
  [1],
  [4],
  [5],
  [0],
  [3],
  [4],
  [2],
  [1],
  [0],
  [1],
  [2],
  [5],
  [4],
  [3],
  [4],
  [5],
  [0],
  [1],
  [2],
  [0],
  [3],
  [4],
  [1],
  [0],
  [2],
  [0],
  [3],
  [5],
  [0],
  [1],
  [4]],
 [[3],
  [1],
  [2],
  [0],
  [2],
  [1],
  [2],
  [5],
  [5],
  [3],
  [2],
  [4],
  [5],
  [5],
  [4],
  [5],
  [

In [56]:
who=numpy.array([1,2,3])
numpy.random.shuffle(who)
print(who)

[2 1 3]


0.26785714285714285